In [1]:
import numpy as np 
import pandas as pd
import torch
import os
from PIL import Image

In [98]:
import torch.nn as nn
from torch.nn import Dropout
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models, transforms
from tqdm.notebook import tqdm

In [21]:
from pathlib import Path

In [22]:
from os.path import dirname, abspath

In [28]:
d = dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname(abspath("loading_dataset.ipynb")))))))))

In [31]:
os.chdir(d + "/MaskedFace-Net")

In [56]:
transform_train = transforms.Compose(
    [transforms.RandomRotation(20),
     transforms.RandomHorizontalFlip(0.5),
     transforms.CenterCrop(300),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [64]:
transform_val = transforms.Compose(
    [transforms.CenterCrop(300),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [57]:
maskedface_net_train = torchvision.datasets.ImageFolder("../MaskedFace-Net/train", transform=transform_train)

In [67]:
maskedface_net_val = torchvision.datasets.ImageFolder("../MaskedFace-Net/validation", transform=transform_val)

In [74]:
maskedface_net_test = torchvision.datasets.ImageFolder("../MaskedFace-Net/holdout", transform=transform_val)

In [61]:
data_loader_train = torch.utils.data.DataLoader(maskedface_net_train,
                                          batch_size=4,
                                          shuffle=True)

In [68]:
data_loader_val = torch.utils.data.DataLoader(maskedface_net_val,
                                             batch_size=4,
                                             shuffle=True)

In [75]:
data_loader_test = torch.utils.data.DataLoader(maskedface_net_test,
                                             batch_size=4,
                                             shuffle=True)

In [79]:
device = torch.device("cuda")

In [90]:
iter(data_loader_train).next()

[tensor([[[[-0.6784, -0.6941, -0.6863,  ...,  0.5451,  0.5529,  0.5451],
           [-0.6784, -0.7098, -0.6941,  ...,  0.5765,  0.5843,  0.5608],
           [-0.6706, -0.7020, -0.6941,  ...,  0.5686,  0.5843,  0.5922],
           ...,
           [ 0.2392,  0.2471,  0.2549,  ...,  0.0118,  0.0510,  0.2314],
           [ 0.2471,  0.2471,  0.2706,  ...,  0.0980,  0.0353,  0.0745],
           [ 0.2784,  0.2549,  0.2706,  ...,  0.3490,  0.2314,  0.0824]],
 
          [[-0.6941, -0.7098, -0.7020,  ...,  0.1843,  0.1765,  0.1765],
           [-0.6941, -0.7255, -0.7098,  ...,  0.2000,  0.2078,  0.1843],
           [-0.6863, -0.7176, -0.7098,  ...,  0.1922,  0.2078,  0.2000],
           ...,
           [ 0.6941,  0.7020,  0.7098,  ...,  0.4196,  0.4039,  0.3882],
           [ 0.7020,  0.7020,  0.7255,  ...,  0.4353,  0.3647,  0.3725],
           [ 0.7333,  0.7098,  0.7255,  ...,  0.4745,  0.4118,  0.4196]],
 
          [[-0.7961, -0.8118, -0.7961,  ...,  0.0667,  0.0667,  0.0510],
           [-

In [10]:
def get_resnet():
    input_img = Input((4, 4, 300,28),name='input_layer')
    zeroPad1 = ZeroPadding2D((1,1), name='zeroPad1', dim_ordering='th')
    zeroPad1_2 = ZeroPadding2D((1,1), name='zeroPad1_2', dim_ordering='th')
    layer1 = Convolution2D(6, 3, 3, subsample=(2, 2), init='he_uniform', name='major_conv', dim_ordering='th')
    layer1_2 = Convolution2D(16, 3, 3, subsample=(2, 2), init='he_uniform', name='major_conv2', dim_ordering='th')
    zeroPad2 = ZeroPadding2D((1,1), name='zeroPad2', dim_ordering='th')
    zeroPad2_2 = ZeroPadding2D((1,1), name='zeroPad2_2', dim_ordering='th')
    layer2 = Convolution2D(6, 3, 3, subsample=(1,1), init='he_uniform', name='l1_conv', dim_ordering='th')
    layer2_2 = Convolution2D(16, 3, 3, subsample=(1,1), init='he_uniform', name='l1_conv2', dim_ordering='th')
    zeroPad3 = ZeroPadding2D((1,1), name='zeroPad3', dim_ordering='th')
    zeroPad3_2 = ZeroPadding2D((1,1), name='zeroPad3_2', dim_ordering='th')
    layer3 = Convolution2D(6, 3, 3, subsample=(1, 1), init='he_uniform', name='l2_conv', dim_ordering='th')
    layer3_2 = Convolution2D(16, 3, 3, subsample=(1, 1), init='he_uniform', name='l2_conv2', dim_ordering='th')
    layer4 = Dense(64, activation='relu', init='he_uniform', name='dense1')
    layer5 = Dense(16, activation='relu', init='he_uniform', name='dense2')
    final = Dense(10, activation='softmax', init='he_uniform', name='classifier')
    
    sgd = SGD(decay=0., lr=0.01, momentum=0.9, nesterov=True)
    model.compile(loss=scc, optimizer=sgd, metrics=['accuracy'])
    return model

## Models

In [100]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [99]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        model_ft = models.resnet18(pretrained=True)
        set_parameter_requires_grad(model_ft, True)
        self.features = nn.Sequential(
            *list(model_ft.children())[:6]
        )
        self.conv1 = nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size = 3)
        self.conv2 = nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size = 3)
        self.conv3 = nn.Conv2d(in_channels = 128, out_channels = 64, kernel_size = 3)
        self.conv4 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3)
        self.bh1 = nn.BatchNorm2d(num_features = 128)
        self.bh2 = nn.BatchNorm2d(num_features = 64)
        self.maxpool = nn.MaxPool2d(2, 2)
        self.dropout = Dropout(p = 0.5)
        self.bh3 = nn.BatchNorm1d(num_features = 500)
        self.lin1 = nn.Linear(in_features = 2304, out_features =500)
        self.lin2 = nn.Linear(in_features = 500, out_features = 1)

    def forward(self, x):
        x = self.features(x)
        x = self.bh1(F.relu(self.conv1(x)))
        x = self.maxpool(x)
        x = self.bh1(F.relu(self.conv2(x)))
        x = self.bh2(F.relu(self.conv3(x)))
        x = self.bh2(F.relu(self.conv4(x)))
        x = self.maxpool(x)
        x = torch.flatten(x, start_dim = 1)
        x = self.dropout(x)
        x = self.bh3(F.relu(self.lin1(x)))
        x = self.lin2(x)  
        return x

In [102]:
model = CNN()

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /tmp/xdg-cache/torch/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 25.0MB/s]


In [103]:
num_epochs = 20

In [105]:
criterion = nn.BCEWithLogitsLoss()

In [109]:
optimizer = optim.Adam(model.parameters(), lr= parameters["learning_rate"])

NameError: name 'parameters' is not defined

## Training

In [104]:
def training(num_epochs, model, optimizer, criterion, train_loader, validation_loader, batch_size, device, save_path):
    train_acc_lst = []
    train_loss_lst = []
    val_acc_lst = []
    val_loss_lst = []
    val_min_loss = np.inf
    
    model = model.double().to(device)
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
        train_loss = 0.0
        val_loss = 0.0

    #training step(we only update the model in the training loop)
        for i, data in enumerate(tqdm(train_loader), 0):
            train_inputs, train_labels = data
            train_inputs, train_labels = train_inputs.double().to(device), train_labels.double().view(-1, 1).to(device)


            optimizer.zero_grad()

            train_outputs = model(train_inputs)
            loss = criterion(train_outputs, train_labels)
            loss.backward()
            optimizer.step()

            #calculate training loss
            train_loss += loss.item() * train_inputs.size(0)

        model.eval()

        for i, data in enumerate(tqdm(validation_loader), 0):
            val_inputs, val_labels = data
            val_inputs, val_labels = val_inputs.double().to(device), val_labels.double().view(-1, 1).to(device)

            val_outputs = model(val_inputs)
            loss = criterion(val_outputs, val_labels)
            val_loss += loss.item() * val_inputs.size(0)

        avg_train_loss = train_loss / (len(train_loader) * batch_size)
        train_acc = (torch.sigmoid(train_outputs).squeeze().round() == train_labels.squeeze()).cpu().numpy().mean()

        avg_val_loss = val_loss / (len(validation_loader) * batch_size)
        val_acc = (torch.sigmoid(val_outputs).squeeze().round() == val_labels.squeeze()).cpu().numpy().mean()


        print("Epoch: {}/{}...".format(epoch+1, num_epochs),
              "train Loss: {:.6f}...".format(avg_train_loss),
              "train accurcy: {:.2f}...".format(train_acc),
              "val Loss: {:.6f}".format(avg_val_loss),
              "val accurcy: {:.2f}...".format(val_acc))

        train_loss_lst.append(avg_train_loss)
        train_acc_lst.append(round(train_acc, 2))
        val_loss_lst.append(avg_val_loss)
        val_acc_lst.append(round(val_acc, 2))

        if avg_val_loss <= val_min_loss:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            val_min_loss,
            avg_val_loss))
            torch.save(model.state_dict(), save_path)
            val_min_loss = avg_val_loss


        model.train()
    
    return train_acc_lst, train_loss_lst, val_acc_lst, val_loss_lst

In [ ]:
train_loss, train_acc, val_loss, val_acc = training(num_epochs, model, optimizer, criterion, train_loader, validation_loader, batch_size, device, save_path)